<a href="https://colab.research.google.com/github/Ivano05/Clasificaci-n-de-Textos/blob/main/Clasificaci%C3%B3n_de_Textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [**Clasificación de opiniones de texto a partir de archivos de texto sin procesar.**](https://)

Setup


In [1]:
import tensorflow as tf
import numpy as np

Cargamos los datos de un sentiment analisis de reseñas de películas de IMDB y examinamos la estructura.



In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  12.4M      0  0:00:06  0:00:06 --:--:-- 16.4M


La carpeta aclImdb contiene una subcarpeta de entrenamiento y prueba:

In [4]:
!ls aclImdb

imdbEr.txt  imdb.vocab	README	test  train


In [5]:
!ls aclImdb/test

labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt


In [6]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


Las carpetas aclImdb/train/pos y aclImdb/train/neg contienen archivos de texto, cada uno de los cuales representa una revisión (ya sea positiva o negativa):

In [7]:
!cat aclImdb/train/pos/6248_7.txt

Being an Austrian myself this has been a straight knock in my face. Fortunately I don't live nowhere near the place where this movie takes place but unfortunately it portrays everything that the rest of Austria hates about Viennese people (or people close to that region). And it is very easy to read that this is exactly the directors intention: to let your head sink into your hands and say "Oh my god, how can THAT be possible!". No, not with me, the (in my opinion) totally exaggerated uncensored swinger club scene is not necessary, I watch porn, sure, but in this context I was rather disgusted than put in the right context.<br /><br />This movie tells a story about how misled people who suffer from lack of education or bad company try to survive and live in a world of redundancy and boring horizons. A girl who is treated like a whore by her super-jealous boyfriend (and still keeps coming back), a female teacher who discovers her masochism by putting the life of her super-cruel "lover" 

Solo nos interesan las subcarpetas pos y neg, así que eliminemos el resto:

In [8]:
!rm -r aclImdb/train/unsup

Puede usar la utilidad tf.keras.preprocessing.text_dataset_from_directory para generar un objeto tf.data.Dataset etiquetado a partir de un conjunto de archivos de texto en el disco archivados en carpetas específicas de clase.

Usémoslo para generar los conjuntos de datos de entrenamiento, validación y prueba. Los conjuntos de datos de validación y entrenamiento se generan a partir de dos subconjuntos del directorio de trenes, con el 20 % de las muestras yendo al conjunto de datos de validación y el 80 % al conjunto de datos de entrenamiento.

Tener un conjunto de datos de validación además del conjunto de datos de prueba es útil para ajustar hiperparámetros, como la arquitectura del modelo, para los cuales no se debe usar el conjunto de datos de prueba.

Sin embargo, antes de poner el modelo en el mundo real, se debe volver a entrenar utilizando todos los datos de entrenamiento disponibles (sin crear un conjunto de datos de validación), para maximizar su rendimiento.

Al usar los argumentos de subconjunto y división_de_validación, asegúrese de especificar una semilla aleatoria o pasar shuffle=False, para que las divisiones de validación y entrenamiento que obtenga no se superpongan.

In [9]:
batch_size = 32
raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="training",
    seed=1337,
)
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

print(f"Number of batches in raw_train_ds: {raw_train_ds.cardinality()}")
print(f"Number of batches in raw_val_ds: {raw_val_ds.cardinality()}")
print(f"Number of batches in raw_test_ds: {raw_test_ds.cardinality()}")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.
Number of batches in raw_train_ds: 625
Number of batches in raw_val_ds: 157
Number of batches in raw_test_ds: 782


Veamos una vista previa de algunas muestras:


In [11]:
# Es importante echar un vistazo a sus datos sin procesar para garantizar su normalización.
# y la tokenización funcionará como se esperaba. Podemos hacer eso tomando algunos
# ejemplos del conjunto de entrenamiento y mirarlos.
# Este es uno de los lugares donde brilla la ejecución ansiosa:
# simplemente podemos evaluar estos tensores usando .numpy()
# en lugar de tener que evaluarlos en un contexto de Sesión/Gráfico.

# It's important to take a look at your raw data to ensure your normalization
# and tokenization will work as expected. We can do that by taking a few
# examples from the training set and looking at them.
# This is one of the places where eager execution shines:
# we can just evaluate these tensors using .numpy()
# instead of needing to evaluate them in a Session/Graph context.
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])

b'I am very disappointed with "K-911." The original "good" quality of "K-9" doesn\'t exist any more. This is more like a sitcom! Some of casts from original movie returned and got some of my memory back. The captain of Dooley now loves to hit him like a scene from old comedy show. That was crazy. What\'s the deal with the change of Police? It seems like they are now LAPD! Not San Diego PD. It is a completely different movie from "'
0
b"Giallo fans, seek out this rare film. It is well written, and full of all sorts of the usual low lifes that populate these films. I don't want to give anything away, so I wont even say anything about the plot. The whole movie creates a very bizarre atmosphere, and you don't know what to expect or who to suspect. Recommended! The only place I've seen to get this film in english is from European Trash Cinema, for $15."
1
b"Terry Gilliam's and David Peoples' teamed up to create one of the most intelligent and creative science fiction movies of the '90's. Pe

Preparacion de datos


En particular, eliminamos las etiquetas <br />.



In [12]:
from tensorflow.keras.layers import TextVectorization
import string
import re

# Después de observar nuestros datos anteriores, vemos que el texto sin procesar contiene una ruptura HTML
# etiquetas de la forma '<br />'. Estas etiquetas no se eliminarán de forma predeterminada
# estandarizador (que no elimina HTML). Debido a esto, tendremos que
# crear una función de estandarización personalizada.

# Having looked at our data above, we see that the raw text contains HTML break
# tags of the form '<br />'. These tags will not be removed by the default
# standardizer (which doesn't strip HTML). Because of this, we will need to
# create a custom standardization function.
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )


# Constantes del modelo.
# Model constants.

max_features = 20000
embedding_dim = 128
sequence_length = 500

# Ahora que tenemos nuestra estandarización personalizada, podemos instanciar nuestro texto
# capa de vectorización. Estamos usando esta capa para normalizar, dividir y mapear
# cadenas a enteros, por lo que establecemos nuestro 'output_mode' en 'int'.
# Tenga en cuenta que estamos usando la función de división predeterminada,
# y la estandarización personalizada definida anteriormente.
# También establecemos una longitud de secuencia máxima explícita, ya que las CNN más adelante en nuestro
# el modelo no admitirá secuencias irregulares.


# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)

# Ahora que se ha creado la capa de vocabulario, llame a `adapt` en un conjunto 
# de datos de solo texto para crear el vocabulario. No tiene que realizar lotes, 
# pero para conjuntos de datos muy grandes, esto significa que no está guardando 
# copias de repuesto del conjunto de datos en la memoria.


# Now that the vocab layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Hagamos un conjunto de datos de solo texto (sin etiquetas):
# Let's make a text-only dataset (no labels):
text_ds = raw_train_ds.map(lambda x, y: x)

# Llamemos a `adaptar`:
# Let's call `adapt`:
vectorize_layer.adapt(text_ds)

Dos opciones para vectorizar los datos Hay 2 formas en que podemos usar nuestra capa de vectorización de texto: Opción 1: Hágalo parte del modelo, para obtener un modelo que procese cadenas en bruto, como este:

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
x = vectorize_layer(text_input)
x = layers.Embedding(max_features + 1, embedding_dim)(x)
...

Opción 2: aplíquelo al conjunto de datos de texto para obtener un conjunto de datos de índices de palabras, luego introdúzcalo en un modelo que espera secuencias enteras como entradas.

Una diferencia importante entre los dos es que la opción 2 le permite realizar procesamiento de CPU asíncrono y almacenamiento en búfer de sus datos cuando entrena en GPU. Entonces, si está entrenando el modelo en GPU, probablemente desee optar por esta opción para obtener el mejor rendimiento. Esto es lo que haremos a continuación.

Si tuviéramos que exportar nuestro modelo a producción, enviaríamos un modelo que acepta cadenas sin formato como entrada, como en el fragmento de código de la opción 1 anterior. Esto se puede hacer después del entrenamiento. Hacemos esto en la última sección.

In [13]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

# Vectorizar los datos.
# Vectorize the data.
train_ds = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)


# Realice una captura previa / 
# almacenamiento en búfer asincrónico de los datos para obtener el mejor rendimiento en la GPU.
# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

Construir un modelo

Elegimos una convnet 1D simple que comienza con una capa de incrustación.

In [14]:
from tensorflow.keras import layers


# Una entrada de número entero para los índices de vocabulario.
# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")


# A continuación, agregamos una capa para mapear esos índices de vocabulario 
# en un espacio de dimensionalidad 'embedding_dim'.
# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + agrupación máxima global
# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)


# Agregamos una capa oculta de vainilla:
# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)


# Proyectamos en una capa de salida de una sola unidad y la aplastamos con un sigmoide:
# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)


# Compile el modelo con pérdida de entropía cruzada binaria y un optimizador Adam.
# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Entrenar el modelo

In [15]:
epochs = 3


# Ajuste el modelo usando los conjuntos de datos de entrenamiento y prueba.
# Fit the model using the train and test datasets.
model.fit(train_ds, validation_data=val_ds, epochs=epochs)

Epoch 1/3
625/625 [==============================] - 122s 193ms/step - loss: 0.4857 - accuracy: 0.7304 - val_loss: 0.3051 - val_accuracy: 0.8762
Epoch 2/3
625/625 [==============================] - 116s 186ms/step - loss: 0.2185 - accuracy: 0.9152 - val_loss: 0.3199 - val_accuracy: 0.8744
Epoch 3/3
625/625 [==============================] - 115s 184ms/step - loss: 0.1178 - accuracy: 0.9563 - val_loss: 0.3790 - val_accuracy: 0.8728


Evaluar el modelo en el set de prueba


In [16]:
model.evaluate(test_ds)

782/782 [==============================] - 34s 44ms/step - loss: 0.3894 - accuracy: 0.8632


[0.3893643319606781, 0.8632400035858154]

Hacer un modelo de punta a punta


Si desea obtener un modelo capaz de procesar cadenas en bruto, simplemente puede
crear un nuevo modelo (usando los pesos que acabamos de entrenar):


In [17]:
# Una entrada de cadena
# A string input
inputs = tf.keras.Input(shape=(1,), dtype="string")

# Convierte cadenas en índices de vocabulario
# Turn strings into vocab indices
indices = vectorize_layer(inputs)

# Convierta los índices de vocabulario en predicciones
# Turn vocab indices into predictions
outputs = model(indices)

# Nuestro modelo de extremo a extremo
# Our end to end model
end_to_end_model = tf.keras.Model(inputs, outputs)
end_to_end_model.compile(
    loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]
)


# Pruébelo con `raw_test_ds`, que produce cadenas sin procesar
# Test it with `raw_test_ds`, which yields raw strings
end_to_end_model.evaluate(raw_test_ds)

782/782 [==============================] - 33s 42ms/step - loss: 0.3894 - accuracy: 0.8632


[0.3893642723560333, 0.8632400035858154]